<a href="https://colab.research.google.com/github/pasdptt/PasdPublicNB/blob/master/Intro2Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Spark on Colab
References Readings/Links <br>
*   http://apache.osuosl.org/spark/spark-3.0.0-preview2/
*   https://medium.com/@sushantgautam_930/apache-spark-in-google-collaboratory-in-3-steps-e0acbba654e6 <br>
*   https://notebooks.gesis.org/binder/jupyter/user/databricks-koalas-kuv5qckt/notebooks/docs/source/getting_started/10min.ipynb <br>
*   https://medium.com/@sushantgautam_930/apache-spark-in-google-collaboratory-in-3-steps-e0acbba654e6 <br>
*   https://towardsdatascience.com/introduction-to-apache-spark-207a479c3001
*   https://spark.apache.org/
*   https://medium.com/@amjadraza24/spark-ifying-pandas-databricks-koalas-with-google-colab-93028890db5

# Step 1 : Install Java, Spark, and Findspark

In [1]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,114 kB]
Hit:14 http://ppa.launch

In [15]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
#Get Install the Java Development Kit
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Get Spark lib -- address need to be change according to the current version of Spark: https://downloads.apache.org/spark/
!wget -q http://apache.osuosl.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
#Unzip th Spark
!tar xf spark-3.3.0-bin-hadoop3.tgz
#install findspark for Python
!pip install -q findspark
#!pip install -q sparksql-magic


In [3]:
#Install extra lib(s)
!pip install -q xlrd
!pip install -q kaggle
!pip install -q kora


     |████████████████████████████████| 57 kB 2.3 MB/s 
     |████████████████████████████████| 1.6 MB 8.1 MB/s 


# Step 2: Set Environment Variables

In [14]:
#import and set environment for spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [5]:
import kora
import pandas as pd

# Step 3: Start a SparkSession

In [16]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import *

SPARK_URL = "local[*]"

spark = SparkSession.builder.master(SPARK_URL).getOrCreate()

In [10]:
# Check the pyspark version
import pyspark
print(pyspark.__version__)

3.3.0


In [17]:
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3)

+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows



In [18]:
df.show(100)

+-----+
|hello|
+-----+
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
|world|
+-----+
only showing top 100 rows



# Special Step: Getting data from kaggle

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
!kaggle datasets download -d hverified/web-scraped-data-of-fifa-world-cup-2018-players

  0% 0.00/13.7k [00:00<?, ?B/s]
100% 13.7k/13.7k [00:00<00:00, 12.3MB/s]


In [ ]:
!unzip /content/web-scraped-data-of-fifa-world-cup-2018-players.zip  && rm /content/web-scraped-data-of-fifa-world-cup-2018-players.zip

Archive:  /content/web-scraped-data-of-fifa-world-cup-2018-players.zip
  inflating: Fifa_players.csv        


# Step 4: Mount CSV data to table

In [20]:
file_path = "./Fifa_players.csv"

df_test = spark.read.options(header="true",inferschema = "true").csv(file_path)

df_test.show()

+---+---+-------+----------+------------------+-------------------+-------------------+----------+
|_c0|Age|Country|Height(cm)|International Caps|International Goals|               Name|      Role|
+---+---+-------+----------+------------------+-------------------+-------------------+----------+
|  0| 29|   Peru|  189.0 cm|                42|                  0|      Pedro GALLESE|Goalkeeper|
|  1| 34|   Peru|  179.0 cm|                75|                  0|  Alberto RODRIGUEZ|  Defender|
|  2| 29|   Peru|  172.0 cm|                24|                  0|         Aldo CORZO|  Defender|
|  3| 27|   Peru|  185.0 cm|                 8|                  0|Anderson SANTAMARIA|  Defender|
|  4| 24|   Peru|  178.0 cm|                 8|                  0|      Miguel ARAUJO|  Defender|
|  5| 26|   Peru|  169.0 cm|                29|                  0|      Miguel TRAUCO|  Defender|
|  6| 28|   Peru|  175.0 cm|                34|                  3|      Paolo HURTADO|Midfielder|
|  7| 27| 

In [21]:
#or

sqlContext = SQLContext(spark)

df_test2 = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load(file_path)

df_test2.show()

/content/spark-3.3.0-bin-hadoop3/python/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


+---+---+-------+----------+------------------+-------------------+-------------------+----------+
|_c0|Age|Country|Height(cm)|International Caps|International Goals|               Name|      Role|
+---+---+-------+----------+------------------+-------------------+-------------------+----------+
|  0| 29|   Peru|  189.0 cm|                42|                  0|      Pedro GALLESE|Goalkeeper|
|  1| 34|   Peru|  179.0 cm|                75|                  0|  Alberto RODRIGUEZ|  Defender|
|  2| 29|   Peru|  172.0 cm|                24|                  0|         Aldo CORZO|  Defender|
|  3| 27|   Peru|  185.0 cm|                 8|                  0|Anderson SANTAMARIA|  Defender|
|  4| 24|   Peru|  178.0 cm|                 8|                  0|      Miguel ARAUJO|  Defender|
|  5| 26|   Peru|  169.0 cm|                29|                  0|      Miguel TRAUCO|  Defender|
|  6| 28|   Peru|  175.0 cm|                34|                  3|      Paolo HURTADO|Midfielder|
|  7| 27| 

In [22]:
df_test.registerTempTable("fifa2018")

/content/spark-3.3.0-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [23]:
sqlContext.sql("""
select * from fifa2018
""").show()

+---+---+-------+----------+------------------+-------------------+-------------------+----------+
|_c0|Age|Country|Height(cm)|International Caps|International Goals|               Name|      Role|
+---+---+-------+----------+------------------+-------------------+-------------------+----------+
|  0| 29|   Peru|  189.0 cm|                42|                  0|      Pedro GALLESE|Goalkeeper|
|  1| 34|   Peru|  179.0 cm|                75|                  0|  Alberto RODRIGUEZ|  Defender|
|  2| 29|   Peru|  172.0 cm|                24|                  0|         Aldo CORZO|  Defender|
|  3| 27|   Peru|  185.0 cm|                 8|                  0|Anderson SANTAMARIA|  Defender|
|  4| 24|   Peru|  178.0 cm|                 8|                  0|      Miguel ARAUJO|  Defender|
|  5| 26|   Peru|  169.0 cm|                29|                  0|      Miguel TRAUCO|  Defender|
|  6| 28|   Peru|  175.0 cm|                34|                  3|      Paolo HURTADO|Midfielder|
|  7| 27| 

In [24]:
sqlContext.sql("""
select Role,count(*) from fifa2018 group by Role
""").show()

+----------+--------+
|      Role|count(1)|
+----------+--------+
|      null|       2|
|  Defender|     246|
|Goalkeeper|      96|
|   Forward|     159|
|Midfielder|     233|
+----------+--------+



In [25]:
sqlContext.sql("""
select Role,avg(age) from fifa2018 group by Role
""").show()

+----------+------------------+
|      Role|          avg(age)|
+----------+------------------+
|      null|              null|
|  Defender|28.304878048780488|
|Goalkeeper|           29.8125|
|   Forward|27.245283018867923|
|Midfielder|27.793991416309012|
+----------+------------------+



# How to save result as CSV to your "Drive"

In [26]:
df_test\
   .coalesce(1)\
   .write.format("com.databricks.spark.csv")\
   .option("header", "true")\
   .save("test_output.csv")\

# Lab 2



1.   Find total unique player in Fifa 2018
2.   Find out that which role (position) paticipate most in Fifa 2018
3.   Find out that in each country, which role (position) paticipate most
4.   Find Maximum and Minimum age in each country players
5.   Find out the average age of each role (position)
6.   How many country paticipate in Fifa 2018
7.   Total Goal of each country by posistion in decending order
8.   Identify how many player(s) who has name as "Miguel", and which country(s) the came from



In [ ]:
#Hint

df_result = sqlContext.sql("""
""").show()

df_test\
   .coalesce(1)\
   .write.format("com.databricks.spark.csv")\
   .option("header", "true")\
   .save()\

+--------+--------------------+
|count(1)|count(DISTINCT Name)|
+--------+--------------------+
|       4|                   4|
+--------+--------------------+



Delivery Point


1.   Results
2.   SQL script(s) to get the result

